# House Prices Prediction EDA

Kaggle API parameters

`COMPETITION_NAME= 'home-data-for-ml-course'`

For downloading competition data

`kaggle competitions download -c home-data-for-ml-course`

For making a submission

`kaggle competitions submit -c home-data-for-ml-course -f sample_submission.csv -m "test submission through kaggle api"`

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Load DataSets

Load training `train.csv` and testing `test.csv` into dataframes.

In [2]:
# Load the data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

print("Dimensions of train: {}".format(train_df.shape))
print("Dimensions of test: {}".format(test_df.shape))

Dimensions of train: (1460, 81)
Dimensions of test: (1459, 80)


## Data Exploration

Peek into the dataset and it's columns

In [5]:
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Missing Values

Check whether the data contains missing values.

In [150]:
print("Number of rows with missing values=", train_df.isnull().any(axis=1).sum())


Number of rows with missing values= 1460


All the rows have missing data.

In [152]:
# Pandas pipeline to check for missing values
pd.DataFrame(train_df.isna().sum()).reset_index()\
.rename(columns={0: 'missing_values'})\
.sort_values(by='missing_values', ascending=False)\
.query('missing_values > 0')\
.pipe(lambda x: x.assign(percentage_missing = x.missing_values / train_df.shape[0] * 100))\
.reset_index()
# .style.hide_index()

,level_0,index,missing_values,percentage_missing
0,72,PoolQC,1453,99.520548
1,74,MiscFeature,1406,96.301370
2,6,Alley,1369,93.767123
3,73,Fence,1179,80.753425
4,25,MasVnrType,872,59.726027
5,57,FireplaceQu,690,47.260274
6,3,LotFrontage,259,17.739726
7,59,GarageYrBlt,81,5.547945
8,64,GarageCond,81,5.547945
9,58,GarageType,81,5.547945


`GarageYrBlt`, `Electrical`, `MasVnrArea` and `LotFrontage` do not have meaning for `NaN`. They are truely missing values. For rest of 14 variables in above table `NaN` means absence of the feature. It shall be indicative of a category. 

## Column Exploration

In [6]:
train_df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

Classify columns into categorical and numerical based on description.txt file.

In [155]:
# Loop over all lines in data_description.txt
cat_cols = list()
num_cols = list()

line_spaces = 0

with open('data_description.txt') as f:
    line = f.readline()
    col = line.split(':')[0] if ':' in line else None
    line = f.readline()
    while line != '':
        new_col = line.split(':')[0] if ':' in line else None
        # # First time assign the column name
        # if col is None and new_col is not None:
        #     col = new_col
        if new_col in train_df.columns:
            if line_spaces > 2:
                cat_cols.append(col)
            else:
                num_cols.append(col)
            # reset line_spaces
            line_spaces = 0
            # set col to new_col
            col = new_col
        else:
            line_spaces += 1

        line = f.readline()

    # Check the last column
    if line_spaces > 1:
        cat_cols.append(col)
    else:
        num_cols.append(col)
        
print("Categorical columns: {}".format(cat_cols))
print("Numerical columns: {}".format(num_cols))
    

Categorical columns: ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'HalfBath', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
Numerical columns: ['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',

Check whether the columns identified and columns in data frame are equal

In [72]:
# Length of cat_cols and num_cols should be equal to the number of columns in the dataframe
print("Num Columns found in description file and train_df are equal:", 
      len(cat_cols) + len(num_cols) == train_df.shape[1])
print(f"Lenghts of column in description={len(cat_cols) + len(num_cols)} and dataframe={train_df.shape[1]}")

Num Columns found in description file and train_df are equal: False
Lenghts of column in description=77 and dataframe=81


In [60]:
# Find the columns that are not in cat_cols or num_cols
set(train_df.columns) - set(cat_cols) - set(num_cols)

{'BedroomAbvGr', 'Id', 'KitchenAbvGr', 'SalePrice'}

`Id` is index column `SalePrice` is output. What does `BedroomAbvGr` and `KitchenAbvGr` columns look like?

Further investigations revealed that `BedroomAbvGr` and `KitchenAbvGr` in dataframe represents *Bedroom* and *Kitchen* description respectively.

These two are ordinal variables so let's add them to category columns

In [61]:
# Add BedroomAbvGr and KitchenAbvGr to cat_cols
cat_cols.append('BedroomAbvGr')
cat_cols.append('KitchenAbvGr')

Examining numerical columns we can identify the following columns can be encoded as ordinal values
- YearBuild
- YearRemodAdd
- BsmtFullBath
- BsmtHalfBath
- FullBath
- HalfBath (already in cat_cols list)
- TotRmsAbvGrd
- Fireplaces
- GarageYrBlt
- GarageCars

Numerical data that shall be encoded with categoreical values
- MoSold
- YrSold


Categorical columns that can use ordinal encoding

- Street (Pave > Grvl)
- Alley (Pave > Grvl)
- LotShape
- Utilities
- Neighborhood
- Condition1
- Condition2
- BldgType
- HouseStyle
- RoofStyle
- RoofMatl
- Exterior1st
- Exterior2nd
- MasVnrType
- ExterQual
- ExterCond
- Foundation
- BsmtQual
- BsmtCond
- BsmtExposure
- BsmtFinType1
- BsmtFinType2
- Heating
- HeatingQC
- CentralAir
- Electrical
- KitchenQual
- Functional
- FireplaceQu
- GarageType
- GarageFinish
- GarageQual
- GarageCond
- PavedDrive
- PoolQC
- Fence
- MiscFeature
- SaleType
- SaleCondition

**Directly used as numerical**
- OverallQual
- OverallCond

All the columns in categorical columns expresses ordinallity. We can clearly see that by grouping based on categories aggregating by taking mean of saleprice.

`train_df.groupby('SaleCondition')[['SalePrice']].mean().sort_values('SalePrice', ascending=False)`


# Results of EDA:
- **Missing Values:** There are 18 columns in total that has missing values. From the 18, in 14 columns `NaN` means absence of the feature as given in description file. In these cases `NaN` shall be treated as a category. 

    - `MiscFeature`
    - `Alley`
    - `Fence`
    - `MasVnrType`
    - `FireplaceQu`
    - `GarageCond`
    - `GarageType`
    - `GarageFinish`
    - `GarageQual`
    - `BsmtFinType2`
    - `BsmtExposure`
    - `BsmtQual`
    - `BsmtCond`
    - `BsmtFinType1`
 
    And in rest of 4 columns `GarageYrBlt`, `Electrical`, `MasVnrArea` and `LotFrontage` do not have meaning for `NaN`. They are truly missing values, these values should be handled by imputation or dropping rows.
- **Columns Exploration:** Columns are broadly separated into two numerical and non-numerical. The non-numerical columns and some numerical columns can be treated by `OrdinalEncoding` as they inherently represent the ordinarility.